# Pembuatan Model Pendeteksian Masker Wajah Menggunakan Resnet50
oleh:
- Elshe Erviana Angely (5025201050)
- Rere Arga Dewanata (5025201078)

## Import Libraries

In [1]:
# Import necessary modules and functions from TensorFlow Keras for building the model and preprocessing images.
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import NumPy and Pandas for data manipulation.
import numpy as np
import pandas as pd

# Import regular expression module for pattern matching.
import re

# Define Model

In [2]:
# Define the number of classes for the classification task.
num_classes = 2

# Create a sequential model, a linear stack of layers.
model = Sequential()

# Add ResNet50 pre-trained on ImageNet without the top classification layer, using average pooling.
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))

# Add a dense layer with softmax activation function for final classification into the specified number of classes.
model.add(Dense(num_classes, activation='softmax'))

# Set the first layer (ResNet50) to be untrainable, meaning its weights will not be updated during training.
model.layers[0].trainable = False


94765736/94765736 [==============================] - 1s 0us/step


# Compile Model

In [3]:
# Compile the model with Stochastic Gradient Descent (SGD) optimizer, categorical crossentropy loss function, and accuracy metric.
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Image Augmentation and Data Preprocessing


In [4]:
# Create an image data generator with data augmentation, including preprocessing input, horizontal flipping, and random shifts in width and height.
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                             horizontal_flip=True,
                                             width_shift_range=0.1,
                                             height_shift_range=0.1)

# Create an image data generator without data augmentation, only including preprocessing input.
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)


# Load Image Dataset

In [5]:
# Import the drive module from the google.colab library to mount Google Drive.
from google.colab import drive

# Mount Google Drive to the '/content/drive/' directory in the Colab environment.
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
#define image size height and width
image_size = 224

dataset_path = "/content/drive/MyDrive/ViskomPengayaan/Dataset"
trainset_path = dataset_path + '/train'
validationset_path = dataset_path + '/val'

classification_classes = ['with_mask','without_mask']

#Prepare training image
train_generator = data_generator_with_aug.flow_from_directory(
                                        directory=trainset_path,
                                        classes = classification_classes,
                                        target_size=(image_size, image_size),
                                        batch_size=10,
                                        class_mode='categorical')
#Prepare validation image
validation_generator = data_generator_no_aug.flow_from_directory(
                                        directory=validationset_path,
                                        classes = classification_classes,
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

Found 1200 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


# Train the Model

In [7]:
# Train the model using data generated batch by batch by train_generator.
# Set the number of steps per epoch to 60, epochs to 4, validation data from validation_generator,
# and validation steps to 1.
fit_stats = model.fit_generator(train_generator,
                                steps_per_epoch=60,
                                epochs=4,
                                validation_data=validation_generator,
                                validation_steps=1)


<ipython-input-7-3360caa50766>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_stats = model.fit_generator(train_generator,


Epoch 1/4
60/60 [==============================] - 156s 2s/step - loss: 0.1542 - accuracy: 0.9467 - val_loss: 0.1659 - val_accuracy: 0.9375
Epoch 2/4
60/60 [==============================] - 140s 2s/step - loss: 0.0647 - accuracy: 0.9733 - val_loss: 0.0300 - val_accuracy: 1.0000
Epoch 3/4
60/60 [==============================] - 135s 2s/step - loss: 0.0340 - accuracy: 0.9900 - val_loss: 0.0890 - val_accuracy: 0.9688
Epoch 4/4
60/60 [==============================] - 137s 2s/step - loss: 0.0350 - accuracy: 0.9883 - val_loss: 0.0354 - val_accuracy: 0.9688


# Test the Model

In [8]:
# Create a test data generator without data augmentation, loading images from the specified directory,
# resizing them to the specified target size, with a batch size of 10, no class mode, and no shuffling.

testset_path = dataset_path + '/test'

test_generator = data_generator_no_aug.flow_from_directory(
    directory=testset_path,
    target_size=(image_size, image_size),
    batch_size=10,
    class_mode=None,
    shuffle=False
)

# Generate predictions for the test data using the trained model, iterating over the test generator
# for a number of steps equal to the length of the test generator, with verbose output.
pred = model.predict_generator(test_generator, steps=len(test_generator), verbose=1)

# Round the predictions class to the nearest integer (0 or 1).
cl = np.round(pred)

# Get the filenames of the test images.
filenames = test_generator.filenames

# Initialize lists to store the real classes and predicted classes.
real_class = []
for file in filenames:
    if re.search("with_mask", file):
        real_class.append(1.0)
    else:
        real_class.append(0.0)

# Extract the predicted class probabilities for class 0 (without mask).
predicted_class = cl[:, 0]

# Create a DataFrame to store the results, including filenames, predicted probabilities, predicted classes,
# and real classes.
results = pd.DataFrame({"file": filenames, "pr": pred[:, 0], "pred_class": predicted_class, "real_class": real_class})


Found 200 images belonging to 2 classes.


<ipython-input-8-35a9e972211f>:16: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = model.predict_generator(test_generator, steps=len(test_generator), verbose=1)


20/20 [==============================] - 46s 2s/step


In [9]:
results

,file,pr,pred_class,real_class
0,with_mask/train_00000859.jpg,0.992490,1.0,1.0
1,with_mask/train_00000860.jpg,0.998714,1.0,1.0
2,with_mask/train_00000861.jpg,1.000000,1.0,1.0
3,with_mask/train_00000862.jpg,0.999953,1.0,1.0
4,with_mask/train_00000863.jpg,0.999999,1.0,1.0
...,...,...,...,...
195,without_mask/53_0_0_20170120223141508.jpg,0.001108,0.0,0.0
196,without_mask/53_0_0_20170120223144035.jpg,0.000095,0.0,0.0
197,without_mask/5_0_3_20170119212110570.jpg,0.039680,0.0,0.0
198,without_mask/5_1_3_20170119151512160.jpg,0.011786,0.0,0.0


## Calculate Model Accuracy

In [10]:
# Calculate the accuracy by counting the number of correct predictions (where the predicted class matches the real class)
# using a generator expression, and dividing by the total number of samples (length of real_class).
res = sum(1 for x, y in zip(real_class, predicted_class) if x == y) / len(real_class)

# Print the calculated accuracy.
print("Accuracy :", res)


Accuracy : 0.995


## Save Model

In [11]:
model.save('MaskDetectionModel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
